<a href="https://colab.research.google.com/github/Tclack88/Lambda/blob/master/DS-1-3-Statistical-Tests-and-Experiments/DS_Unit_1_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 3

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 is a variable that indicates whether or not a person's request for credit has been approved or denied. This is a good candidate for a y variable since we might want to use the other features to predict this one. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

Hint: If a column has the datatype of "object" even though it's made up of float or integer values, you can coerce it to act as a numeric column by using the `pd.to_numeric()` function.

In [0]:
import pandas as pd
import numpy as np
from scipy import stats

In [0]:
labels = ["A"+str(i) for i in range(1,17)]  # create header names

data = "https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"
df = pd.read_csv(data,names = labels,na_values="?")

#df.isna().sum()  # returns 0 but source tells us there are missing values, thus there is a non-standard encoding for nan values
#df.A1.unique()   # a '?' is present, this must be the missing value, so we add na_values="?"

df.head(5)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

## T-Tests

In [0]:
# app = approved dataframe
# dis = disapproved dataframe
app = df[df.A16 == '+']
dis = df[df.A16 == '-']

In [0]:
numeric_cols = df.select_dtypes(exclude = 'object').columns.to_list()  # get list of numeric columns

def apply_judgement(pvalue,col):
  if pvalue > .05:
    print("\nNO statistically significant difference between approval rate and",col)
  elif pvalue <.05:
    print("\nSTATISTICALLY SIGNIFICANT difference between approval rate and",col)
    
    
# compare numeric column data    
print('Comparing Attributes between approved and dissaproved loans for numeric data')
for col in numeric_cols:
  print("\n\n")
  print(col)
  t,p = stats.ttest_ind(app[col],dis[col],nan_policy='omit')
  print("t-statistic:",t,"\tp-value:",p)
  apply_judgement(p,col)
  print("\n")

Comparing Attributes between approved and dissaproved loans for numeric data



A2
t-statistic: 4.2922156166315535 	p-value: 2.0276637071781407e-05

STATISTICALLY SIGNIFICANT difference between approval rate and A2





A3
t-statistic: 5.52998337614816 	p-value: 4.551680702308068e-08

STATISTICALLY SIGNIFICANT difference between approval rate and A3





A8
t-statistic: 8.935819983773698 	p-value: 3.6710537401601785e-18

STATISTICALLY SIGNIFICANT difference between approval rate and A8





A11
t-statistic: 11.667004222431277 	p-value: 7.957718568079967e-29

STATISTICALLY SIGNIFICANT difference between approval rate and A11





A14
t-statistic: -2.6358251986645476 	p-value: 0.008586135473979569

STATISTICALLY SIGNIFICANT difference between approval rate and A14





A15
t-statistic: 4.680216020964486 	p-value: 3.4520256956287944e-06

STATISTICALLY SIGNIFICANT difference between approval rate and A15




## chi squared tests

In [0]:
object_cols = df.select_dtypes('object').columns.to_list()
object_df = df[object_cols]


# Large p-values imply similarity, for there to be distinction in the selection criteria,
# there needs to be a small p-value, meaning there's a small chance for these to randomly coincide


for col in object_cols[:-1]:
  print('\n\n\n',col)
  ct = pd.crosstab(object_df[col],object_df.A16)
  ct['All'] = ct.sum(axis=1)
  ct.loc['All'] = ct.sum()
  print(ct)
  chi_square,p_value,dof,expected = stats.chi2_contingency(ct)
  print("chi squared value:",chi_square,"\tpvalue:",p_value)
  apply_judgement(p_value,col)





 A1
A16    +    -  All
A1                
a     98  112  210
b    206  262  468
All  304  374  678
chi squared value: 0.41143508194591916 	pvalue: 0.981530018361231

NO statistically significant difference between approval rate and A1



 A4
A16    +    -  All
A4                
l      2    0    2
u    256  263  519
y     45  118  163
All  303  381  684
chi squared value: 26.234074966202144 	pvalue: 0.0002013603203375773

STATISTICALLY SIGNIFICANT difference between approval rate and A4



 A5
A16    +    -  All
A5                
g    256  263  519
gg     2    0    2
p     45  118  163
All  303  381  684
chi squared value: 26.234074966202144 	pvalue: 0.0002013603203375773

STATISTICALLY SIGNIFICANT difference between approval rate and A5



 A6
A16    +    -  All
A6                
aa    19   35   54
c     62   75  137
cc    29   12   41
d      7   23   30
e     14   11   25
ff     7   46   53
i     14   45   59
j      3    7   10
k     14   37   51
m     16   22   38
q     51   27

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

---

## My interpretation:
Numeric columns were columns: A2, A3, A8, A11, A14, A15 and ALL of them proved to be statistically significant in determining approval rate. This conclusion is based on t-tests and their associated p-values, none of which were above .05 (A14 however was really close, with a p-value of .008, nearly within the top 1% range of its associated t-distribution)

Non-Numeric columns were: A1, A4, A5, A6, A7, A9, A10, A12 and A13. In this, A1, A12 and A13 proved to be statstically INSIGNIFICANT in determining loan approval rates. This was determined through chi-squared testing. A1, A12 then A13 with pvalues of .98, .95 and .16 respectively which means the "probability" of them appearing similar is rather high. We can see by looking at any of the crosstabs, there are strong similarities in the ratios moving down the columns of the approved(+) and disaproved(-) for A1,A12 and A13.

### Conclusion:
The features: A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A14, A15 appear to be the best predictors of loan approval. eyeballing the crosstabs, we can see some of those:

ex. in A4, y tends to result in loan disapproval

The assignment was challenging because there's little to no context with what each category means. Some categories have a "t" and "f" which may be True and False, but we still can't gather what this may be referring to. This may be helpful in unbiasing the analysis, however it removes some "common sense checks" which help vet the analysis. I almost reached the opposite conclusion for the chi-squared test until I realized that low p-values imply low degree of similarity

## Stretch Goals

Please do not work on anything listed in this section until you have adequately answered everything in the above three sections, all of these stretch goals are **OPTIONAL** but completing them will ensure you a score of 3 on the corresponding sections.

### Section 1 Stretch Goal: (do the following)
- Go above and beyond in conducting thorough data exploration including high quality comments and/or markdown text cells to explain your process/discoveries.

### Section 2 Stretch Goals: (do one of the following)
 - Write a function(s) to automate the execution of t-tests on the continuous variables treating the different class labels as the two samples.

 - Write a funciton(s) to automate the execution of chi^2 tests on all of the different combinations of categorical variables in the dataset.

### Section 3 Stretch Goals: (do one of the following)
- Construct a confidence interval around the mean of one of the continuous variables, communicate the results/interpretation of that confidence interval in the most consumable/easy-to-understand way that you can. (You may choose to include a graph here, but you don't necessarily have to) Communicate the precision of your estimate as clearly as possible.

- Explain the relationship between confidence intervals and T-tests, if possible, use code to demonstrate some important aspects of te